In [1]:
%load_ext autoreload
%autoreload 2
import sys
    
from utils.store_data import ReadData
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
import pandas as pd
from answer_judge.vqaEval import VQAEval
import numpy as np

In [2]:
data_reader=ReadData('/root/autodl-tmp/hallscope/pope/adversarial/train')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)
df.head()

,img_path,question,label,question_id,category,most_likely,responses
0,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a person in the image?\nAnswer the q...,[tensor(0)],[tensor(9686)],[adversarial],{'embedding': [[[ 0.00099182 0.0020752 -0.00...,None
1,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a cup in the image?\nAnswer the ques...,[tensor(0)],[tensor(4690)],[adversarial],{'embedding': [[[ 0.00099182 0.0020752 -0.00...,None
2,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a chair in the image?\nAnswer the qu...,[tensor(0)],[tensor(4706)],[adversarial],{'embedding': [[[ 0.00099182 0.0020752 -0.00...,None
3,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a frisbee in the image?\nAnswer the ...,[tensor(1)],[tensor(4943)],[adversarial],{'embedding': [[[ 0.00099182 0.0020752 -0.00...,None
4,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a chair in the image?\nAnswer the qu...,[tensor(0)],[tensor(10024)],[adversarial],{'embedding': [[[ 0.00099182 0.0020752 -0.00...,None


In [4]:
df.iloc[0]['most_likely']['response']

['No']

In [5]:
df['response']=df['most_likely'].apply(lambda x: x['response'][0])
resps=df['response'].to_list()
resps[:5]

['No', 'No', 'No', 'Yes', 'No']

In [6]:
df['img_path']=df['img_path'].apply(lambda x: x[0])

In [7]:
df['label']=df['label'].apply(lambda x: x[0].item())
gt_label=df['label'].to_list()
gt_label[:5]

[0, 0, 0, 1, 0]

In [8]:
df['question']=df['question'].apply(lambda x: x[0])

In [9]:
df = df.drop(columns=['most_likely','responses','question_id','label'])

In [10]:
is_correct=[]
for i,x in enumerate(resps):
    if x.lower()=='no' and gt_label[i]==0:
        is_correct.append(1)
    elif x.lower()=='yes' and gt_label[i]==1:
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [11]:
df['is_correct']=is_correct

In [12]:
df.head()

,img_path,question,category,response,is_correct
0,/root/autodl-fs/coco_images/train/COCO_train20...,Is there a person in the image?\nAnswer the qu...,[adversarial],No,1
1,/root/autodl-fs/coco_images/train/COCO_train20...,Is there a cup in the image?\nAnswer the quest...,[adversarial],No,1
2,/root/autodl-fs/coco_images/train/COCO_train20...,Is there a chair in the image?\nAnswer the que...,[adversarial],No,1
3,/root/autodl-fs/coco_images/train/COCO_train20...,Is there a frisbee in the image?\nAnswer the q...,[adversarial],Yes,1
4,/root/autodl-fs/coco_images/train/COCO_train20...,Is there a chair in the image?\nAnswer the que...,[adversarial],No,1


In [13]:
import json

with open('./pope_adversarial_resp.json', 'w') as file:
    json.dump(df.to_dict(orient='records'), file, ensure_ascii=False, indent=4)